In [93]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import json

In [2]:
data = pd.read_csv('./Junction-Kesko-Receipt-Data/Junction_data.csv', sep=';',
                  parse_dates=['TransactionDate'], dtype={'Quantity': 'float64'}, decimal=',')

In [3]:
data.shape

(52941708, 10)

In [4]:
data.head()

,AreaId,Receipt,TransactionDate,BeginHour,EAN,Quantity,PersonAgeGrp,KCustomer,QualClass,EasyClass
0,1,356601823178935,2017-11-07,17,5410103915654,1.0,55-64,6715,Q_1-3,E_4-7
1,1,356341113181337,2017-11-05,12,6413466126704,1.0,35-44,6712,Q_1-3,E_4-7
2,1,356629240622521,2017-11-07,18,24000017677,1.0,45-54,6715,Q_1-3,E_4-7
3,1,356269217607293,2017-11-04,13,6412000033188,2.0,55-64,6712,Q_4-7,E_4-7
4,1,356215448049286,2017-11-04,11,2000940900000,1.0,45-54,6713,Q_1-3,E_8-10


In [5]:
data.dtypes

AreaId                      int64
Receipt                     int64
TransactionDate    datetime64[ns]
BeginHour                   int64
EAN                         int64
Quantity                  float64
PersonAgeGrp               object
KCustomer                   int64
QualClass                  object
EasyClass                  object
dtype: object

In [242]:
data_young = data[(data['PersonAgeGrp'] == '18-24') & \
                  (data['QualClass'] == 'Q_1-3') & \
                  (data['EasyClass'] == 'E_8-10') & \
                  (data['AreaId'] == 2) & \
                  (data['TransactionDate'] >= '2019-08-01')]

In [243]:
data_old = data[(data['PersonAgeGrp'] == '25-34') & \
                (data['QualClass'] == 'Q_8-10') & \
                (data['EasyClass'] == 'E_1-3') & \
                (data['AreaId'] == 2) & \
                (data['TransactionDate'] >= '2019-08-01')]

In [244]:
np.random.seed(0)
receipts_young = np.random.choice(data_young['Receipt'].unique(), size=40, replace=False)
data_young = data_young[data_young['Receipt'].isin(receipts_young)]

receipts_old = np.random.choice(data_old['Receipt'].unique(), size=40, replace=False)
data_old = data_old[data_old['Receipt'].isin(receipts_old)]

In [248]:
data_young = data_young.copy()
data_old = data_old.copy()

In [250]:
data_young['user_id'] = 'young'
data_old['user_id'] = 'old'

In [251]:
data_subset = pd.concat([data_young, data_old], ignore_index=True)

In [253]:
data_subset.drop(columns=['AreaId', 'BeginHour', 'PersonAgeGrp', 'KCustomer', 'QualClass', 'EasyClass'], inplace=True)

In [254]:
data_subset.head()

,Receipt,TransactionDate,EAN,Quantity,user_id
0,446828127191328,2019-08-03,6408640995924,8.0,young
1,446828127191328,2019-08-03,6415600551090,1.0,young
2,446828127191328,2019-08-03,6430064401125,1.0,young
3,446828127191328,2019-08-03,2000973900008,1.0,young
4,446828127191328,2019-08-03,6413600167501,2.0,young


In [255]:
data_subset.to_csv('data_subset.csv', index=False)

In [221]:
data_young['TransactionDate'].min()

Timestamp('2017-11-01 00:00:00')

In [222]:
data_young['TransactionDate'].max()

Timestamp('2019-10-31 00:00:00')

In [226]:
data_young['Receipt'].nunique()

4804

In [227]:
data_old['Receipt'].nunique()

2379

In [230]:
receipts_young

array([450721792349421, 455109585418071, 450696221676072, 454085320960924,
       446616206118797, 455454208253237, 449062736261979, 455366760071529,
       449806440520771, 449441961383308, 453184953409073, 446828127191328,
       449871144596011, 453422297275771, 451274647265067, 457433950162526,
       456144012159158, 455014417787804, 455248974137488, 459249888532901,
       449025388836783, 446539185471684, 457564040576294, 452775945997572,
       447412527187983, 454566596584255, 453423426615653, 448653144544667,
       455019974628145, 459394773410055])

In [7]:
data_young.shape

(383038, 10)

In [11]:
data_young['Receipt'].nunique()

71541

In [22]:
data.groupby(['AreaId'])['Receipt'].nunique()

AreaId
1    1297364
2    2106076
3    1329669
4    1418564
Name: Receipt, dtype: int64

In [10]:
data_old.shape

(780536, 10)

In [12]:
data_old['Receipt'].nunique()

96914

In [26]:
data_young.groupby(['KCustomer'])['Receipt'].nunique()

KCustomer
6711    23590
6712     1174
6714     2915
Name: Receipt, dtype: int64

In [24]:
data_old.groupby(['KCustomer'])['Receipt'].nunique()

KCustomer
6711    7558
6712    6251
6713    2501
6714     518
Name: Receipt, dtype: int64

In [27]:
data_young.groupby(['KCustomer'])['EAN'].nunique()

KCustomer
6711    14985
6712     2256
6714     3102
Name: EAN, dtype: int64

In [29]:
data_young_eans = set(data_young['EAN'])
data_old_eans = set(data_old['EAN'])

In [31]:
len(data_young_eans | data_old_eans)

26010

In [32]:
eans = list(data_young_eans | data_old_eans)

In [35]:
eans_selected = [ean for ean in eans if not str(ean).startswith('2')]

In [50]:
ean_value_counts = data['EAN'].value_counts()

In [54]:
popular_eans = ean_value_counts.iloc[:5000].index
popular_eans = [ean for ean in popular_eans if not str(ean).startswith('2')]

In [58]:
len(popular_eans)

4584

In [37]:
len(eans_selected)

23971

In [38]:
import requests

In [49]:
ean_to_dict = {}

for ean in tqdm(eans_selected[:3000]):
    response = requests.post(url='https://kesko.azure-api.net/v1/search/products',
                  headers={'Ocp-Apim-Subscription-Key': '50d5b4ef0f664d94b115f23c6da5b3e7',
                           'Content-Type': 'application/json'},
                  json={"filters": {
                            "ean": str(ean)
                        },
                        "view": {
                            "offset": 0,
                            "limit": 10,
                            "showFacets": {
                                "facets": [
                                    "string"
                                ],
                                "limit": 250
                            },
                            "showAvailability": {
                                "storeAvailability": [
                                    "A208",
                                    "A210",
                                    "A212",
                                    "A214",
                                    "A216",
                                    "A218",
                                    "A300",
                                    "A301",
                                    "A302",
                                    "A303",
                                    "A304",
                                    "A305",
                                    "A306",
                                    "A308",
                                    "A310",
                                    "A205",
                                    "A311",
                                    "A313",
                                    "A314"
                                ],
                                "webstoreAvailability": []
                            }
                        }
                       }
                 )
    ean_to_dict[str(ean)] = response.json()

In [67]:
pd.Series([json['totalHits'] if 'totalHits' in json else -1 for ean, json in ean_to_dict.items()]).value_counts()

 1    2395
 0     603
-1       2
dtype: int64

In [74]:
[(ean , json) for ean, json in ean_to_dict.items() if 'totalHits' not in json or json['totalHits'] == 0][:5]

[('6430037770302', {'totalHits': 0, 'results': []}),
 ('5765228396636', {'error': 'Processing Error.'}),
 ('6430037770333', {'totalHits': 0, 'results': []}),
 ('7315340001376', {'totalHits': 0, 'results': []}),
 ('3600523591831', {'totalHits': 0, 'results': []})]

In [209]:
products_df = pd.DataFrame([{'ean': ean,
  'isAlcohol': json['results'][0]['isAlcohol'],
  'pictureUrl': json['results'][0]['pictureUrls'][0]['original'] if json['results'][0]['pictureUrls'] else None,
  'isConsumerGood': json['results'][0]['isConsumerGood'] if 'isConsumerGood' in json['results'][0] else None,
  'marketingName_finnish': json['results'][0]['marketingName']['finnish'] if 'finnish' in json['results'][0]['marketingName'] else None,
  'marketingName_english': json['results'][0]['marketingName']['english'] if 'english' in json['results'][0]['marketingName'] else None,
  'ingredients_finnish': json['results'][0]['attributes']['MATERIAL_U']['value']['value'] if 'MATERIAL_U' in json['results'][0]['attributes'] else None,
  'ingredients_english': json['results'][0]['attributes']['MATERIAL_E']['value']['value'] if 'MATERIAL_E' in json['results'][0]['attributes'] else None,
  'net_weight': json['results'][0]['measurements']['netWeight'] if 'measurements' in json['results'][0] else None,
  
                             
  'proteins': json['results'][0]['attributes']['PROTEG']['value']['value'] if 'PROTEG' in json['results'][0]['attributes'] else None,
  'fats': json['results'][0]['attributes']['RASVAA']['value']['value'] if 'RASVAA' in json['results'][0]['attributes'] else None,
  'carbohydrates': json['results'][0]['attributes']['HIHYDR']['value']['value'] if 'HIHYDR' in json['results'][0]['attributes'] else None,
  
  'kcal': json['results'][0]['attributes']['ENERKC']['value']['value'] if 'ENERKC' in json['results'][0]['attributes'] else None,
  'fats_saturated': json['results'][0]['attributes']['TYYDRH']['value']['value'] if 'TYYDRH' in json['results'][0]['attributes'] else None,
  'tags': json['results'][0]['attributes']['TX_RAVOMI']['value'] if 'TX_RAVOMI' in json['results'][0]['attributes'] else None,
  

                             
} for ean, json in ean_to_dict.items() if 'totalHits' in json and json['totalHits'] == 1])



In [210]:
len(products_df)

2395

In [217]:
with open('tags_sample.json', 'wt', encoding='utf-8') as file:
    file.write(json.dumps(products_df['tags'].iloc[:1000].to_dict()).encode().decode('unicode-escape'))

In [202]:
res = products_df['ingredients_finnish'].str.upper().str.extractall(r"(?P<e_number>\bE\d{3,4}\b)")
res['count'] = 1
res.set_index('e_number', append=True, drop=True, inplace=True)

res = res.unstack('e_number', fill_value=0)
res = res.groupby(level=0, as_index=True, group_keys=False).sum()
res = res.droplevel(0, axis=1)
res = res.sort_index(axis=1)

In [203]:
products_with_e = pd.concat([products_df, res], axis=1)

In [204]:
products_with_e[res.columns] = products_with_e[res.columns].fillna(0).astype(int)

In [205]:
products_with_e[res.columns]

,E100,E101,E1105,E120,E1200,E129,E131,E132,E133,E140,...,E901,E903,E904,E950,E951,E954,E955,E965,E967,E999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2391,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2392,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2393,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [258]:
products_with_e.head()

,ean,isAlcohol,pictureUrl,isConsumerGood,marketingName_finnish,marketingName_english,ingredients_finnish,ingredients_english,net_weight,proteins,...,E901,E903,E904,E950,E951,E954,E955,E965,E967,E999
0,6412000034819,False,https://k-file-storage-qa.imgix.net/f/k-ruoka/...,False,Saarioinen pinaattiohukaiset puolukkahillolla ...,None,"Pinaattiohukaiset 170 g: VEHNÄjauho, vesi, pi...",None,0.20,4.8,...,0,0,0,0,0,0,0,0,0,0
1,6407970095106,False,https://k-file-storage-qa.imgix.net/f/k-ruoka/...,False,Korpela lihapyörykät 350g gluteeniton,Korpela meatballs 350g gluten-free,"Sianliha (Suomi), vesi, muunnettu tärkkelys (t...",None,0.35,12.0,...,0,0,0,0,0,0,0,0,0,0
2,6412000034840,False,https://k-file-storage-qa.imgix.net/f/k-ruoka/...,False,Saarioinen riisipuuro 270g laktoositon,None,"Laktoositon MAITOjuoma, riisi, vesi, laktoosit...",None,0.27,2.7,...,0,0,0,0,0,0,0,0,0,0
3,5701259100187,False,https://k-file-storage-qa.imgix.net/f/k-ruoka/...,False,Dansukker Taloussokeri 1kg,None,Sokeri,Sugar,1.00,0.0,...,0,0,0,0,0,0,0,0,0,0
4,6411401035807,False,https://k-file-storage-qa.imgix.net/f/k-ruoka/...,False,Fazerin Parhain 220g pussi,None,"sokeri, glukoosisiirappi, MAITO,kasvirasva (pa...","sugar, glucose syrup, MILK, vegetable fat (pal...",0.22,1.6,...,0,0,0,0,0,0,0,0,0,0


In [266]:
e_count = products_with_e[[col for col in products_with_e.columns if col.startswith('E')]].sum(axis=0)

res = e_count.sort_values(ascending=False).iloc[:10].to_dict()

In [257]:
res.columns.values

array(['E100', 'E101', 'E1105', 'E120', 'E1200', 'E129', 'E131', 'E132',
       'E133', 'E140', 'E141', 'E1414', 'E1420', 'E1422', 'E1442', 'E150',
       'E1520', 'E1521', 'E153', 'E160', 'E162', 'E163', 'E170', 'E171',
       'E172', 'E200', 'E202', 'E211', 'E212', 'E216', 'E220', 'E221',
       'E223', 'E224', 'E235', 'E250', 'E251', 'E252', 'E260', 'E261',
       'E262', 'E263', 'E270', 'E281', 'E282', 'E296', 'E300', 'E301',
       'E304', 'E306', 'E315', 'E316', 'E320', 'E322', 'E325', 'E326',
       'E327', 'E330', 'E331', 'E333', 'E334', 'E336', 'E337', 'E338',
       'E339', 'E340', 'E341', 'E385', 'E401', 'E402', 'E404', 'E405',
       'E406', 'E407', 'E410', 'E412', 'E414', 'E415', 'E418', 'E420',
       'E422', 'E440', 'E442', 'E444', 'E445', 'E450', 'E451', 'E452',
       'E460', 'E461', 'E463', 'E464', 'E466', 'E471', 'E473', 'E475',
       'E476', 'E481', 'E492', 'E494', 'E499', 'E500', 'E503', 'E508',
       'E509', 'E535', 'E551', 'E570', 'E575', 'E579', 'E621', 'E627'

In [208]:
products_with_e.to_csv('products_v2.csv', index=False)

In [112]:
res.head(10)

e_number
   match         
1  0         E450
   1         E322
4  0         E330
   1         E331
   2         E420
   3         E471
   4         E141
   5         E120
10 0         E330
   1         E300

In [118]:
res['e_number'].nunique()

125

In [100]:
products_df.to_csv('products_v1.csv', index=False)

In [65]:
for key, value in ean_to_dict.items():
    print(value['totalHits'])
    break

1


In [94]:
with open('eans_sample.json', 'wt') as file:
    json.dump(ean_to_dict, file)

In [43]:
{
    
} for ean, json in ean_to_dict.items()

{'totalHits': 1,
 'results': [{'urlSlug': 'korpela-lihapyorykat-350g-gluteeniton-6407970095106',
   'isAlcohol': False,
   'pictureUrls': [{'original': 'https://k-file-storage-qa.imgix.net/f/k-ruoka/product/6407970095106'}],
   'isConsumerGood': False,
   'description': {},
   'type': 'kRuoka',
   'marketingName': {'swedish': 'Korpela köttbullar 350g glutenfri',
    'finnish': 'Korpela lihapyörykät 350g gluteeniton',
    'english': 'Korpela meatballs 350g gluten-free'},
   'ean': '6407970095106',
   'segment': {'finnish': 'Pyörykät, pihvit, puikot, kääryleet', 'id': '1310'},
   'popularity': 0,
   'attributes': {'HOEHE': {'value': {'type': 'decimal', 'value': 11.4}},
    'LANDX': {'value': {'type': 'string', 'value': 'Suomi'}},
    'NTGEW': {'value': {'type': 'decimal', 'value': 0.35}},
    'ZZVENDOR': {'value': {'type': 'string',
      'explanation': {'finnish': 'LIHAJALOSTE KORPELA OY'},
      'value': '66149'}},
    'ENERKC': {'value': {'type': 'decimal', 'value': 240}},
    'MATERI

In [36]:
eans_selected[:10]

[6412000034819,
 6407970095106,
 6412000034840,
 5701259100187,
 6411401035807,
 5701092114471,
 6412000034857,
 6411300634682,
 6408641052732,
 6430037770302]

In [28]:
data_old.groupby(['KCustomer'])['EAN'].nunique()

KCustomer
6711    12870
6712    10553
6713     8140
6714     2119
Name: EAN, dtype: int64

In [17]:
data_young.head()

,AreaId,Receipt,TransactionDate,BeginHour,EAN,Quantity,PersonAgeGrp,KCustomer,QualClass,EasyClass
7434,1,420047458681112,2019-01-30,8,6412000034789,1.0,18-24,6711,Q_1-3,E_8-10
7578,1,420047458681112,2019-01-30,8,6408430023608,1.0,18-24,6711,Q_1-3,E_8-10
7722,1,420047458681112,2019-01-30,8,6408430024858,2.0,18-24,6711,Q_1-3,E_8-10
7866,1,420047458681112,2019-01-30,8,6411401017575,1.0,18-24,6711,Q_1-3,E_8-10
71,1,420193976441735,2019-01-30,10,6408641827200,2.0,18-24,6711,Q_1-3,E_8-10


In [20]:
data_young['KCustomer'].value_counts()

6711    422
6714     54
6712     24
Name: KCustomer, dtype: int64

In [16]:
data_old.shape

(0, 10)

In [12]:
data.sort_values(['AreaId', 'PersonAgeGrp', 'KCustomer', 'QualClass', 'EasyClass',
                  'TransactionDate', 'BeginHour', 'Receipt'], inplace=True)

In [13]:
data.head(30)

,AreaId,Receipt,TransactionDate,BeginHour,EAN,Quantity,PersonAgeGrp,KCustomer,QualClass,EasyClass
12091,1,420220933163461,2019-01-30,13,6413600017189,1.000,18-24,6711,Q_1-3,E_1-3
12235,1,420220933163461,2019-01-30,13,5701259100187,2.000,18-24,6711,Q_1-3,E_1-3
12379,1,420220933163461,2019-01-30,13,6411600081162,5.000,18-24,6711,Q_1-3,E_1-3
12523,1,420220933163461,2019-01-30,13,6411401033919,1.000,18-24,6711,Q_1-3,E_1-3
12667,1,420220933163461,2019-01-30,13,6408641827200,3.000,18-24,6711,Q_1-3,E_1-3
12811,1,420220933163461,2019-01-30,13,6415600501811,1.000,18-24,6711,Q_1-3,E_1-3
12955,1,420220933163461,2019-01-30,13,6410405154170,1.000,18-24,6711,Q_1-3,E_1-3
13099,1,420220933163461,2019-01-30,13,7310240651287,1.000,18-24,6711,Q_1-3,E_1-3
13243,1,420220933163461,2019-01-30,13,7310791180663,2.000,18-24,6711,Q_1-3,E_1-3
13387,1,420220933163461,2019-01-30,13,6430041692775,2.000,18-24,6711,Q_1-3,E_1-3
